In [1]:
# 원하는 손글씨 숫자를 생성하기
# 이렇게 흑백을 컬러로, 선화를 채색 

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10


In [3]:
### 모델 구성
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈와 실제 이미지에 그에 해당하는 숫자에 대한 정보를 넣어주기 위해 사용함
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise값에 labels 정보를 추가
        inputs = tf.concat([noise, labels], 1)
        
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
        
    return output

def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기위해
        # 이전에 사용되었던 변수를 재사용하도록 한다.
        if reuse:
            scope.reuse_variables()
            
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation = None)
        
    return output

def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])



In [4]:
# 생성 모델과 판별 모델에 Y 즉, label 정보를 추가하여 labels 정보에 이미지 생성할 수 있도록 유도

G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# ------------------------------------------------------------
# 손실함수는 다음을 참고하여 GAN 논문에 나온 방식과는 약간 다르게 작성하였습니다.
# http://bamos.github.io/2016/08/09/deep-completion/
# 진짜 이미지를 판별하는 D_real 값은 1에 가깝도록,
# 가짜 이미지를 판별하는 D_gene 값은 0에 가깝도록 하는 손실 함수입니다.
# ------------------------------------------------------------

loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                                logits = D_real, labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                                logits=D_gene, labels = tf.zeros_like(D_gene)))

# loss_D_real 과 loss_D_gene 를 더한뒤 이 값을 최소화 하도록 한다.
loss_D = loss_D_real + loss_D_gene
# 가짜 이미지를 진짜에 가깝게 만들도록 생성망을 학습하기 위해 D_gene를 최대한 1에 가깝도록 만드는 손실함수
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                            logits=D_gene, labels=tf.ones_like(D_gene)))


# discriminator 와 generator scope 에서 사용된 변수들을 가져온다.
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                          scope = 'discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                          scope = 'generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)


In [7]:
## 모델 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0,0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], 
                                feed_dict = {X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], 
                                 feed_dict = {Y: batch_ys, Z: noise})
        
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    ## 이미지 생성 저장
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size],
                                        Z: noise})
        
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range (sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        
print('끝!')

Epoch: 0000 D loss: 0.003695 G loss: 8.15
Epoch: 0001 D loss: 0.007697 G loss: 7.578
Epoch: 0002 D loss: 0.00251 G loss: 10.02
Epoch: 0003 D loss: 0.01371 G loss: 10.49
Epoch: 0004 D loss: 0.0009083 G loss: 9.377
Epoch: 0005 D loss: 0.0001888 G loss: 10.56
Epoch: 0006 D loss: 0.00329 G loss: 10.74
Epoch: 0007 D loss: 0.0001798 G loss: 10.9
Epoch: 0008 D loss: 1.672e-05 G loss: 14.87
Epoch: 0009 D loss: 3.172e-06 G loss: 14.99
Epoch: 0010 D loss: 7.383e-07 G loss: 15.0
Epoch: 0011 D loss: 5.097e-05 G loss: 11.6
Epoch: 0012 D loss: 0.0001007 G loss: 14.3
Epoch: 0013 D loss: 0.001125 G loss: 20.3
Epoch: 0014 D loss: 1.547e-05 G loss: 12.59
Epoch: 0015 D loss: 8.358e-06 G loss: 12.66
Epoch: 0016 D loss: 8.26e-08 G loss: 18.93
Epoch: 0017 D loss: 1.771e-06 G loss: 13.73
Epoch: 0018 D loss: 1.908e-06 G loss: 14.54
Epoch: 0019 D loss: 1.297e-05 G loss: 12.81
Epoch: 0020 D loss: 1.523e-06 G loss: 14.39
Epoch: 0021 D loss: 4.258e-06 G loss: 14.65
Epoch: 0022 D loss: 4.977e-07 G loss: 16.28
Epoc